<a href="https://colab.research.google.com/github/DeboraMendes/supermarkets/blob/main/supermarkets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [299]:
# Instalando biblioteca do Google Maps
!pip install -q googlemaps

# Instalando biblioteca do Pandas
!pip install -q pandas

In [300]:
# Importando biblioteca para integrar com Google Maps
import googlemaps

# Importando biblioteca para criação e manipulação de DataFrames (tabelas)
import pandas as pd

#Importando biblioteca para ter acesso aos Secrets do Colab
from google.colab import userdata

#Importando biblioteca para tratar a apresentação dos dados
from IPython.display import display, HTML

# Importando biblioteca para tempo de espera antes de solicitar a próxima página
import time

In [301]:
# Configurando chave da API do Google Maps
maps_api_key = userdata.get('MAPS_API_KEY')

In [302]:
# Configurando o cliente do Google Maps
gmaps = googlemaps.Client(key=maps_api_key)

In [303]:
# Função para buscar supermercados em uma cidade com paginação
def search_supermarkets_in_city(city):
    try:
        results = []
        page_token = None
        page_count = 1  # Contador de página

        while True:
            places_result = gmaps.places(
                query=f"supermarkets in {city}",
                type='supermarket',
                page_token=page_token
            )
            results.extend(places_result['results'])
            page_token = places_result.get('next_page_token')

            if not page_token:
                print("Não existem mais páginas.")
                break
            else:
                print(f"Buscando dados da página {page_count}...")
                page_count += 1
                time.sleep(5)

        return results
    except googlemaps.exceptions.ApiError as e:
        print(f"API Error: {e}")
        return None
    except Exception as e:
        print(f"Other Error: {e}")
        return None

In [304]:
# Função para buscar detalhes do lugar, incluindo o telefone
def search_place_details(place_id):
    try:
        place_details = gmaps.place(place_id=place_id, fields=['name', 'formatted_address', 'formatted_phone_number'])
        return place_details['result']
    except googlemaps.exceptions.ApiError as e:
        print(f"API Error: {e}")
        return None
    except Exception as e:
        print(f"Other Error: {e}")
        return None


In [305]:
# Função para buscar supermercados e retornar os dados em formato de tabela
def get_supermarkets_table(city):
    supermarkets = search_supermarkets_in_city(city)
    if not supermarkets:
        print(f"Nenhum supermercado encontrado em {city}.")
        return pd.DataFrame(columns=["Número", "Nome", "Endereço", "Telefone"])

    data = []
    for i, supermarket in enumerate(supermarkets, 1):
        details = search_place_details(supermarket['place_id'])
        if details:
            nome = details.get('name', 'N/A')
            endereco = details.get('formatted_address', 'N/A')
            telefone = details.get('formatted_phone_number', 'N/A')
            data.append([i, nome, endereco, telefone])
        else:
            data.append([i, 'N/A', 'N/A', 'N/A'])

    df = pd.DataFrame(data, columns=["Número", "Nome", "Endereço", "Telefone"])
    return df

In [306]:
# Exemplo de uso da função para obter a tabela de supermercados
city = "Pato Branco"
supermarkets_table = get_supermarkets_table(city)

Buscando dados da página 1...
Não existem mais páginas.


In [307]:
# Exibir a tabela com rolagem horizontal
html = supermarkets_table.to_html(index=False)
display(HTML(f'<div style="overflow-x: auto; white-space: nowrap;">{html}</div>'))

Número,Nome,Endereço,Telefone
1,Super Polo Supermarkets,"R. Guarani, 640 - Centro, Pato Branco - PR, 85501-050, Brazil",(46) 3225-2599
2,Center Center Center Supermarkets and Fashions,"R. Ibiporã, 805 - Centro, Pato Branco - PR, 85501-282, Brazil",(46) 3025-9400
3,Center Avenida,"Av. Tupi, 150 - Bortot, Pato Branco - PR, 85504-000, Brazil",(46) 3122-9200
4,Comprão - Atacado & Varejo,"Av. Tupi, 6225 - Santo Antônio, Pato Branco - PR, 85501-120, Brazil",(46) 3122-9300
5,Supermercado Planalto,"R. Édimo Belmiro Pastro, 532 - Planalto I, Pato Branco - PR, 85509-280, Brazil",(46) 3225-2166
6,Supermercado Centro Sul,"Av. Tupi, 5773 - Santo Antônio, Pato Branco - PR, 85507-000, Brazil",(46) 3223-2671
7,Patão Supermercado,"Av. Tupi, 3233 - Baixada, Pato Branco - PR, 85505-141, Brazil",(46) 3220-3800
8,Mercado Vitória,"R. José Fraron, 250 - Fraron, Pato Branco - PR, 85503-320, Brazil",(46) 3225-8444
9,Compre Mais Pato Branco - Zona Sul,"Av. Tupi, 5110 - Cristo Rei, Pato Branco - PR, 85507-513, Brazil",(46) 3223-3101
10,No Ponto Supermercados - Zona Norte,"Av. Tupi, 759 - Bortot, Pato Branco - PR, 85504-250, Brazil",(46) 2604-1423
